This code removes the background of every image in the given input directory and then stores it into output directory

In [1]:
# Importing Required Modules 
from rembg import remove, new_session
from PIL import Image 
import os

input_directory = "test_dataset_gray"
output_directory = "test_dataset_no_back"

# removal algorithm options
#["u2net", "u2netp", "u2net_human_seg", "u2net_cloth_seg", "silueta", "isnet-general-use"]
my_session = new_session("isnet-general-use")

# remove background from every image in input_directory and store them in output_directory
for filename in os.listdir(input_directory):
    f = os.path.join(input_directory, filename)

    # Store path of the output image in the variable output_path 
    output_path = os.path.join(output_directory, filename)

    # Processing the image 
    input = Image.open(f) 

    # Removing the background from the given Image 
    output = remove(input, session=my_session)

    # convert to RGB 
    output_rgb = output.convert('RGB')

    #Saving the image in the given path
    output_rgb.save(output_path)